## Trump Model Training

In [70]:
from tensorflow import keras
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
import numpy as np
from tensorflow.keras.utils import to_categorical
from datetime import datetime
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import Adagrad, Adam
from tensorflow.keras import regularizers
import tensorflow as tf


In [71]:
## define paths to data files
path_to_train_data = Path("C:\\Users\\matth\\Documents\\DL4G\\jass-data\\split\\train\\filtered\\card\\csv")
path_to_test_data = Path("C:\\Users\\matth\\Documents\\DL4G\\jass-data\\split\\test\\filtered\\card\\csv")
path_to_val_data = Path("C:\\Users\\matth\\Documents\\DL4G\\jass-data\\split\\val\\filtered\\card\\csv")

In [72]:
# train data
data_train1 = pd.read_csv(path_to_train_data / '0001.csv', header=None)
data_train2 = pd.read_csv(path_to_train_data / '0002.csv', header=None)
data_train3 = pd.read_csv(path_to_train_data / '0003.csv', header=None)
data_train4 = pd.read_csv(path_to_train_data / '0004.csv', header=None)
data_val1 = pd.read_csv(path_to_val_data / '0001.csv', header=None)
data_val2 = pd.read_csv(path_to_val_data / '0002.csv', header=None)

data_train = pd.concat([data_train1, data_train2, data_train3, data_train4, data_val1, data_val2], axis=0)
#data_val = pd.concat([data_val1, data_val2], axis=0)

In [73]:
# test data
data_test1 = pd.read_csv(path_to_test_data / '0001.csv', header=None)
data_test2 = pd.read_csv(path_to_train_data / '0002.csv', header=None)

data_test = pd.concat([data_test1, data_test2])
data_test.shape
data_test.head()

,0,1,2,3,4,5,6,7,8,9,...,73,74,75,76,77,78,79,80,81,82
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,21
1,1,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,20
2,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,12
3,0,0,0,1,0,1,0,1,0,0,...,0,0,1,0,0,0,0,0,1,3
4,0,1,0,0,0,1,0,0,1,0,...,0,0,1,0,0,0,1,0,0,8


## Label data for readability

## Prepare x and y values

In [74]:
# colums used for extracting x and y values. The same effect could be achieved with train_test_split-Method, but 
# since we already have different files, we dont need to split the files using this method.
#data_X_columns = cards + forehand
#data_Y_colums = trump

x_train = data_train[data_train.columns[0:82]]
y_train = data_train[data_train.columns[82]]
#print (x_train.head())
#print (y_train.head())

x_test = data_test[data_test.columns[0:82]]
y_test = data_test[data_test.columns[82]]

## Create model and train it

In [75]:
# We have 37 features, so we have a node for each feature. There are 7 output categories: each trump color(4), 
# obe-abe, unde-ufe, schiebe. So we need an reducing function with 7 elements
model = keras.Sequential()
model.add(keras.layers.Dense(82, activation='relu', input_shape=[82]))
model.add(keras.layers.Dense(82, activation='relu'))
model.add(keras.layers.Dense(82, activation='relu', kernel_regularizer=regularizers.l2(0.01), activity_regularizer=regularizers.l2(0.01)))
model.add(keras.layers.Dense(82, activation='relu'))
model.add(keras.layers.Dense(82, activation='relu'))
model.add(keras.layers.Dense(82, activation='relu'))
model.add(keras.layers.Dense(36, activation='softmax'))

opt = Adam()

model.compile(loss='categorical_crossentropy',
              optimizer= 'adam',
              metrics=['accuracy'])

print(model.summary())


Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_66 (Dense)             (None, 82)                6806      
_________________________________________________________________
dense_67 (Dense)             (None, 82)                6806      
_________________________________________________________________
dense_68 (Dense)             (None, 82)                6806      
_________________________________________________________________
dense_69 (Dense)             (None, 82)                6806      
_________________________________________________________________
dense_70 (Dense)             (None, 82)                6806      
_________________________________________________________________
dense_71 (Dense)             (None, 82)                6806      
_________________________________________________________________
dense_72 (Dense)             (None, 36)               

In [ ]:
y_train_categorical = to_categorical(y_train)

callbacks = [EarlyStopping(monitor='val_loss', patience=2),
             ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)]

history = model.fit(x_train, y_train_categorical, validation_split=0.20, epochs=150, batch_size=128, callbacks=callbacks)

Train on 408132 samples, validate on 102033 samples
Epoch 1/150
408132/408132 [==============================] - 15s 36us/sample - loss: 1.9281 - accuracy: 0.4238 - val_loss: 1.5377 - val_accuracy: 0.4990
Epoch 2/150
408132/408132 [==============================] - 15s 37us/sample - loss: 1.4542 - accuracy: 0.5206 - val_loss: 1.4073 - val_accuracy: 0.5324
Epoch 3/150
408132/408132 [==============================] - 16s 40us/sample - loss: 1.3558 - accuracy: 0.5437 - val_loss: 1.3135 - val_accuracy: 0.5517
Epoch 4/150
408132/408132 [==============================] - 21s 51us/sample - loss: 1.2980 - accuracy: 0.5564 - val_loss: 1.3079 - val_accuracy: 0.5466
Epoch 5/150
408132/408132 [==============================] - 16s 39us/sample - loss: 1.2554 - accuracy: 0.5666 - val_loss: 1.2474 - val_accuracy: 0.5694
Epoch 6/150
408132/408132 [==============================] - 15s 36us/sample - loss: 1.2215 - accuracy: 0.5760 - val_loss: 1.2191 - val_accuracy: 0.5754
Epoch 7/150
408132/408132 [===

Epoch 53/150
408132/408132 [==============================] - 14s 35us/sample - loss: 0.8879 - accuracy: 0.6430 - val_loss: 0.9237 - val_accuracy: 0.6307
Epoch 54/150
408132/408132 [==============================] - 15s 37us/sample - loss: 0.8859 - accuracy: 0.6439 - val_loss: 0.9271 - val_accuracy: 0.6301
Epoch 55/150
408132/408132 [==============================] - 16s 38us/sample - loss: 0.8857 - accuracy: 0.6432 - val_loss: 0.9132 - val_accuracy: 0.6315
Epoch 56/150
408132/408132 [==============================] - 16s 39us/sample - loss: 0.8828 - accuracy: 0.6444 - val_loss: 0.9287 - val_accuracy: 0.6323
Epoch 57/150
408132/408132 [==============================] - 13s 33us/sample - loss: 0.8826 - accuracy: 0.6440 - val_loss: 0.9229 - val_accuracy: 0.6291
Epoch 58/150
408132/408132 [==============================] - 13s 32us/sample - loss: 0.8806 - accuracy: 0.6447 - val_loss: 0.9410 - val_accuracy: 0.6288
Epoch 59/150
408132/408132 [==============================] - 13s 32us/sampl

408132/408132 [==============================] - 15s 37us/sample - loss: 0.8402 - accuracy: 0.6608 - val_loss: 0.8838 - val_accuracy: 0.6464
Epoch 107/150
408132/408132 [==============================] - 15s 38us/sample - loss: 0.8391 - accuracy: 0.6602 - val_loss: 0.8833 - val_accuracy: 0.6461
Epoch 108/150
408132/408132 [==============================] - 15s 36us/sample - loss: 0.8400 - accuracy: 0.6599 - val_loss: 0.8834 - val_accuracy: 0.6471
Epoch 109/150
408132/408132 [==============================] - 14s 34us/sample - loss: 0.8390 - accuracy: 0.6608 - val_loss: 0.8820 - val_accuracy: 0.6472
Epoch 110/150
408132/408132 [==============================] - 15s 37us/sample - loss: 0.8386 - accuracy: 0.6616 - val_loss: 0.8929 - val_accuracy: 0.6413
Epoch 111/150
408132/408132 [==============================] - 15s 38us/sample - loss: 0.8397 - accuracy: 0.6607 - val_loss: 0.8740 - val_accuracy: 0.6472
Epoch 112/150
408132/408132 [==============================] - 14s 34us/sample - los

## Generate graphs for loss and accuracy

In [ ]:
plt.plot(history.history['loss'])
plt.title('Loss')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Accuracy')
plt.xlabel('epoch')
plt.ylabel('acc')
plt.legend(['Train', 'Val'], loc='upper left')

## Test model

In [ ]:
y_test_categorical = to_categorical(y_test)
model.evaluate(x_test, y_test_categorical)

In [ ]:
model.save("card_model_e120_b256_sgd.h5")